<a href="https://colab.research.google.com/github/AnnLivio/powerbi/blob/main/Pizza_sales_sqlite.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# We ♥ Pizza: a Pizza Sales Analysis 🍕

### **Orders** Table

|Field|Description|
|---|---|
|order\_id|Unique identifier|
|date|Date the order was placed|
|time|Time the order was placed|



### **Order details** Table
|Field|Description|
|---|---|
|order\_details\_id| Unique identifier for each pizza|
|order\_id| Foreign key, ties the details in each order to the order |
|pizza\_id| Foreign key, ties the pizza ordered to its details |
|quantity| Quantity ordered for each pizza, same type and size|



### **Pizza** Table
|Field|Description|
|---|---|
|**pizza\_id**| Unique identifier for each pizza |
|**pizza\_type\_id**| Foreign key, ties each pizza to its pizza type |
| **size** |Size of the pizza |
|**price** |Price of the pizza in USD|



### **Pizza_type** Table

|Field|Description|
|---|---|
|**pizza_type_id**| Unique identifier for each pizza type|
|**name**| Name of the pizza as shown in the menu|
|**category**| Category in the menu (Classic, Chicken, Supreme, or Veggie)|
|**ingredients** | Comma-delimited ingredients as shown in the menu|

## Import libraries

In [1]:
# Import libraries and packages
import pandas as pd
import sqlite3

## Create a database, connection and cursor

In [2]:
con = sqlite3.connect('pizzasales.db')
cur = con.cursor()

In [4]:
%load_ext sql

In [5]:
%sql sqlite:///pizzasales.db

In [8]:
# My salvation!!!
%config SqlMagic.style = '_DEPRECATED_DEFAULT'

## Create tables

In [6]:
table_list = ['orders', 'order_details', 'pizzas', 'pizza_types']

for table in table_list:
  path = f"/content/drive/MyDrive/Data Analysis/Simple_Projects/pizza_sales_data/{table}.csv"
  df = pd.read_csv(path, encoding="latin-1")
  df.to_sql(table, con, if_exists='replace', index=False, method=None)
  print(table)


orders
order_details
pizzas
pizza_types


## Look tables

In [9]:
%sql pragma table_info(orders)

 * sqlite:///pizzasales.db
Done.


cid,name,type,notnull,dflt_value,pk
0,order_id,INTEGER,0,None,0
1,date,TEXT,0,None,0
2,time,TEXT,0,None,0


In [ ]:
%sql pragma table_info(order_details)

 * sqlite:///pizzasales.db
Done.


cid,name,type,notnull,dflt_value,pk
0,order_details_id,INTEGER,0,None,0
1,order_id,INTEGER,0,None,0
2,pizza_id,TEXT,0,None,0
3,quantity,INTEGER,0,None,0


In [ ]:
%sql pragma table_info(pizza_types)

 * sqlite:///pizzasales.db
Done.


cid,name,type,notnull,dflt_value,pk
0,pizza_type_id,TEXT,0,None,0
1,name,TEXT,0,None,0
2,category,TEXT,0,None,0
3,ingredients,TEXT,0,None,0


In [ ]:
%sql pragma table_info(pizzas)

 * sqlite:///pizzasales.db
Done.


cid,name,type,notnull,dflt_value,pk
0,pizza_id,TEXT,0,None,0
1,pizza_type_id,TEXT,0,None,0
2,size,TEXT,0,None,0
3,price,REAL,0,None,0


# Orders and Order_details tables

In [ ]:
%sql SELECT * FROM orders LIMIT 5

 * sqlite:///pizzasales.db
Done.


order_id,date,time
1,2015-01-01,11:38:36
2,2015-01-01,11:57:40
3,2015-01-01,12:12:28
4,2015-01-01,12:16:31
5,2015-01-01,12:21:30


In [ ]:
%sql SELECT strftime('%d',date) FROM orders LIMIT 5

 * sqlite:///pizzasales.db
Done.


"strftime('%d',date)"
01
01
01
01
01


# Pizzas tables

## Create a view

In [ ]:
CREATE VIEW pizzas_full AS
SELECT
  pizza_id,
  price,
  size,
  pt.name,
  pt.category
FROM pizzas
INNER JOIN pizza_types pt ON pt.pizza_type_id = pizzas.pizza_type_id;

## CTEs

In [10]:
# Create view table for pizzas and pizza types
%%sql
WITH pizzasfull AS (
  SELECT
    pizza_id,
    price,
    size,
    pt.name,
    pt.category
  FROM pizzas
  INNER JOIN pizza_types pt ON pt.pizza_type_id = pizzas.pizza_type_id
  ),
  ordersfull AS (
    SELECT
      od.order_id,
      date,
      strftime('%H', time) AS hour,
      od.pizza_id,
      od.quantity
    FROM orders
    INNER JOIN order_details od ON od.order_id = orders.order_id
  )
SELECT order_id, date, hour, quantity, pizzasfull.price, pizzasfull.size, pizzasfull.name, pizzasfull.category
FROM ordersfull
LEFT JOIN pizzasfull USING(pizza_id)
LIMIT 5
;

 * sqlite:///pizzasales.db
Done.


order_id,date,hour,quantity,price,size,name,category
1,2015-01-01,11,1,13.25,M,The Hawaiian Pizza,Classic
2,2015-01-01,11,1,16.0,M,The Classic Deluxe Pizza,Classic
2,2015-01-01,11,1,18.5,L,The Five Cheese Pizza,Veggie
2,2015-01-01,11,1,20.75,L,The Italian Supreme Pizza,Supreme
2,2015-01-01,11,1,16.0,M,The Mexicana Pizza,Veggie


## **Questions**

### 1 How many customers do we have each day? Are there any peak hours?

In [11]:
# How many customers do we have each day?
%sql SELECT ROUND(AVG(customers),0) FROM (SELECT COUNT(order_id) AS customers FROM orders GROUP BY date);

 * sqlite:///pizzasales.db
Done.


"ROUND(AVG(customers),0)"
60.0


In [48]:
# Are there any peak hours?
%%sql
SELECT
    strftime('%H', time) AS hour,
    SUM(od.quantity) AS total_qty
  FROM orders
  INNER JOIN order_details od ON od.order_id = orders.order_id
  GROUP BY hour
  ORDER BY total_qty DESC LIMIT 5;

 * sqlite:///pizzasales.db
Done.


hour,total_qty
12,6776
13,6413
18,5417
17,5211
19,4406


In [49]:
# Off-peak hours
%%sql
SELECT
    strftime('%H', time) AS hour,
    SUM(od.quantity) AS total_qty
  FROM orders
  INNER JOIN order_details od ON od.order_id = orders.order_id
  GROUP BY hour
  ORDER BY total_qty LIMIT 5;

 * sqlite:///pizzasales.db
Done.


hour,total_qty
09,4
10,18
23,68
22,1386
21,2545


### 2. How many pizzas are typically in an order? Do we have any bestsellers?

In [ ]:
# Average pizzas per order
%%sql
WITH fullorders AS (
  SELECT
      od.order_id,
      SUM(od.quantity) AS total_qty
  FROM orders
  INNER JOIN order_details od ON od.order_id = orders.order_id
  GROUP BY od.order_id
)
SELECT ROUND(AVG(total_qty),2) as 'AVG Quantity' FROM fullorders;

 * sqlite:///pizzasales.db
Done.


AVG Quantity
2.32


In [52]:
# Bestseller pizza
%%sql
WITH fullpizzas AS (
  SELECT
    pizza_id,
    pt.name as name
  FROM pizzas
  INNER JOIN pizza_types pt ON pt.pizza_type_id = pizzas.pizza_type_id
)
SELECT
    fp.name,
    SUM(quantity) AS total_qty
  FROM order_details od
  INNER JOIN fullpizzas fp ON fp.pizza_id = od.pizza_id
  GROUP BY fp.name
  ORDER BY total_qty DESC LIMIT 5;


 * sqlite:///pizzasales.db
Done.


name,total_qty
The Classic Deluxe Pizza,2453
The Barbecue Chicken Pizza,2432
The Hawaiian Pizza,2422
The Pepperoni Pizza,2418
The Thai Chicken Pizza,2371


In [43]:
# Total pizza (name, no matter size) ordered
%%sql
WITH fullpizzas AS (
  SELECT
    pizza_id,
    pt.name as name
  FROM pizzas
  INNER JOIN pizza_types pt ON pt.pizza_type_id = pizzas.pizza_type_id
)
SELECT
    count(distinct(fp.name))
  FROM order_details od
  INNER JOIN fullpizzas fp ON fp.pizza_id = od.pizza_id
;

 * sqlite:///pizzasales.db
Done.


count(distinct(fp.name))
32


### 3. How much money did we make this year? Can we indentify any seasonality in the sales?

In [ ]:
# How much money did we make this year?
%%sql
WITH subtotals AS (
  SELECT
    od.pizza_id,
    (od.quantity * p.price) AS subtotal

  FROM order_details od, pizzas p
  WHERE od.pizza_id = p.pizza_id
  )
  SELECT ROUND(SUM(subtotal),2) AS year_revenue FROM subtotals;

 * sqlite:///pizzasales.db
Done.


year_revenue
817860.05


In [ ]:
# Can we indentify any seasonality in the sales?
%%sql
WITH sales_per_month AS (
  SELECT strftime('%m', date) as month, COUNT(order_id) as total_orders
  FROM orders
  GROUP BY month
)
SELECT CASE month
          WHEN '01' THEN 'January'
          WHEN '02' THEN 'February'
          WHEN '03' THEN 'March'
          WHEN '04' THEN 'April'
          WHEN '05' THEN 'May'
          WHEN '06' THEN 'June'
          WHEN '07' THEN 'July'
          WHEN '08' THEN 'August'
          WHEN '09' THEN 'September'
          WHEN '10' THEN 'October'
          WHEN '11' THEN 'November'
           ELSE 'December'
       END AS month,
       total_orders
FROM sales_per_month
ORDER BY total_orders DESC;

 * sqlite:///pizzasales.db
Done.


month,total_orders
July,1935
May,1853
January,1845
August,1841
March,1840
April,1799
November,1792
June,1773
February,1685
December,1680


### 4. Are there any pizzas we should take of the menu, or any promotions we could leverage?

In [51]:
# worstseller pizza
%%sql
WITH fullpizzas AS (
  SELECT
    pizza_id,
    pt.name as name
  FROM pizzas
  INNER JOIN pizza_types pt ON pt.pizza_type_id = pizzas.pizza_type_id
)
SELECT
    fp.name,
    SUM(quantity) AS total_qty
  FROM order_details od
  INNER JOIN fullpizzas fp ON fp.pizza_id = od.pizza_id
  GROUP BY fp.name
  ORDER BY total_qty LIMIT 10;

 * sqlite:///pizzasales.db
Done.


name,total_qty
The Brie Carre Pizza,490
The Mediterranean Pizza,934
The Calabrese Pizza,937
The Spinach Supreme Pizza,950
The Soppressata Pizza,961
The Spinach Pesto Pizza,970
The Chicken Pesto Pizza,973
The Italian Vegetables Pizza,981
The Chicken Alfredo Pizza,987
The Green Garden Pizza,997


In [33]:
# Can we indentify any seasonality in the sales?
# count(distinct(strftime('%d', date))) we count the distinct day with orders
%%sql
WITH sales_per_month AS (
  SELECT strftime('%m', date) as month, COUNT(order_id) as total_orders, COUNT(order_id) / count(distinct(strftime('%d', date))) as avg_orders
  FROM orders
  GROUP BY month
)
SELECT CASE month
          WHEN '01' THEN 'January'
          WHEN '02' THEN 'February'
          WHEN '03' THEN 'March'
          WHEN '04' THEN 'April'
          WHEN '05' THEN 'May'
          WHEN '06' THEN 'June'
          WHEN '07' THEN 'July'
          WHEN '08' THEN 'August'
          WHEN '09' THEN 'September'
          WHEN '10' THEN 'October'
          WHEN '11' THEN 'November'
           ELSE 'December'
       END AS month,
       total_orders, avg_orders
FROM sales_per_month
ORDER BY total_orders DESC;

 * sqlite:///pizzasales.db
Done.


month,total_orders,avg_orders
July,1935,62
May,1853,59
January,1845,59
August,1841,59
March,1840,59
April,1799,59
November,1792,59
June,1773,59
February,1685,60
December,1680,56


In [31]:
# Can we indentify any seasonality in the sales?
%%sql
WITH sales_per_month AS (
  SELECT strftime('%m', date) as month, COUNT(order_id) as orders, count(distinct(strftime('%d', date))) as num_days
  FROM orders
  GROUP BY month
)
SELECT CASE
          WHEN month IN('01','02','03') THEN 'Q1'
          WHEN month IN('04','05','06') THEN 'Q2'
          WHEN month IN('07','08','09') THEN 'Q3'
          ELSE 'Q4'
       END AS quarter,
       SUM(orders) AS total_orders, MAX(orders), MIN(orders), SUM(orders) / SUM(num_days) AS avg_orders
FROM sales_per_month
GROUP BY quarter
ORDER BY quarter;

 * sqlite:///pizzasales.db
Done.


quarter,total_orders,MAX(orders),MIN(orders),avg_orders
Q1,5370,1845,1685,59
Q2,5425,1853,1773,59
Q3,5437,1935,1661,60
Q4,5118,1792,1646,58


## **Observaciones**
+ La pizza menos popular es la "Brie Carre Pizza" con aproximadamente un 50% menos de ventas que la "Mediterranean Pizza", la segunda con menos ventas.
+ Las pizzas más populares son: "Classic Deluxe Pizza", "Barbecue Chicken Pizza" y "Hawaiian Pizza".
+ Se observan mayores ventas entre las 12 y 14 hs.
+ El total de ventas anual entre de 9 a 11 y a las 23 en particular, son notablemente menores.
+ En cuanto a la estacionalidad, podemos observar leve crecimiento de ventas en Julio con un total de 1935 unidades vendidas y un promedio de 62 diarias. Octubre es el mes con menos ventas pero hemos podido
+ En términos de cuatrimestres, el último del año es el que presenta menores ventas.

**Acciones**
+ Podría evaluarse el rendimiento por valor de la "Brie Carre" para considerar su retirada del menú o no.
+ Evaluar la apertura al público a partir de las 10 y establecer el cierre a las 22.
+ La "Pepperoni Pizza" está entre las 5 más vendidas mientras que la "Calabrese Pizza" figura entre las 5 menos vendidas. Esta es una oportunidad maravillosa para publicitar ambas y favorecer a la menos afortunada.
+ Algo similar sucede con las pizzas con pollo, ofrecer descuentos o recomendaciones sobre las menos vendidas a los clientes que compran las que están en el top 5.


In [53]:
con.close()

## Visualizaciones en Power BI.

Analysis by Ann Livio.